In [ ]:
!pip install dwave-ocean-sdk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.4/160.4 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.7/103.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.6/225.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.8 MB/s eta 0:00:00


In [ ]:
import itertools
import math
import random
import matplotlib.pyplot as plt
import numpy as np

from dimod import BinaryQuadraticModel, Vartype, SampleSet
from dwave.system import EmbeddingComposite, DWaveSampler
# from dimod.reference.samplers import ExactSolver  # optional

############################################################
# 1. Eingaben: Team, Hierarchie, Interaktionen (Konstant)
############################################################

team_members = ["P1", "P2", "P3", "P4", "P5", "P6", "P7"]
big_five_traits = ["O", "C", "E", "A", "N"]

# Beispielhafte "Ausgangs-Neigungen" (Lineare Biases)
personality_bias = {
    ("P1","O"):  0.9,  ("P1","C"): -0.8,  ("P1","E"):  0.8,  ("P1","A"): -0.5,  ("P1","N"):  0.0,
    ("P2","O"): -0.9,  ("P2","C"):  0.7,  ("P2","E"):  0.8,  ("P2","A"):  0.5,  ("P2","N"): -0.2,
    ("P3","O"): -0.1,  ("P3","C"):  0.9,  ("P3","E"): -0.2,  ("P3","A"):  0.5,  ("P3","N"):  0.1,
    ("P4","O"):  0.4,  ("P4","C"): -0.3,  ("P4","E"):  0.6,  ("P4","A"):  0.2,  ("P4","N"):  0.6,
    ("P5","O"): -0.6,  ("P5","C"):  0.2,  ("P5","E"):  0.4,  ("P5","A"): -0.1,  ("P5","N"): -0.1,
    ("P6","O"):  0.0,  ("P6","C"):  0.8,  ("P6","E"): -0.8,  ("P6","A"): -0.8,  ("P6","N"):  0.2,
    ("P7","O"):  0.2,  ("P7","C"):  0.0,  ("P7","E"): -0.2,  ("P7","A"): -0.2,  ("P7","N"): -0.1,
}

hierarchy_qubits = {
    "ProjectGoal": ["G"],
    "Epics":        ["E1", "E2", "E3"],
    "Features_E1": ["F1_1", "F1_2", "F1_3"],
    "Features_E2": ["F2_1", "F2_2", "F2_3"],
    "Features_E3": ["F3_1", "F3_2", "F3_3"]
}

# c) Wechselwirkungen (etwas willkürlich gewählt)
hierarchy_interaction_strength = 0.01
epic_feature_interaction_strength = 0.01
epic_epic_coupling = 0.05
feature_feature_coupling = 0.02

personality_hierarchy_couplings = {
    ("O","ProjectGoal"):  0.8,
    ("O","Epics"):        0.5,
    ("O","Features"):     0.0,
    ("C","ProjectGoal"): -0.5,
    ("C","Epics"):       -0.1,
    ("C","Features"):     0.6,
    ("E","Epics"):        0.4,
    ("E","Features"):     0.2,
    ("A","Epics"):        0.2,
    ("N","ProjectGoal"): -0.1
}

personality_personality_couplings = {
    ("O","O"):  0.2,
    ("C","C"): -0.1,
    ("E","E"): -0.2,
    ("A","A"):  0.6,
    ("N","N"): -0.1,
    ("O","E"):  0.5,
    ("C","E"):  0.0,
    ("A","N"):  0.0,
    ("O","C"): -0.5
}

############################################################
# Hilfslisten
############################################################

hierarchy_vars = []
for _, qubit_list in hierarchy_qubits.items():
    hierarchy_vars.extend(qubit_list)

all_personality_vars = [
    f"{p}_{t}" for p in team_members for t in big_five_traits
]
all_vars = all_personality_vars + hierarchy_vars

goal_vars = ["G"]
epics_vars = ["E1","E2","E3"]
features_vars = (hierarchy_qubits["Features_E1"]
                 + hierarchy_qubits["Features_E2"]
                 + hierarchy_qubits["Features_E3"])


############################################################
# 2. build_bqm(...) mit param. scale_factor => Kopplungen
############################################################

from dimod import BinaryQuadraticModel, Vartype

def build_bqm(scale_factor):
    bqm = BinaryQuadraticModel({}, {}, 0.0, Vartype.BINARY)

    def add_linear(var, bias):
        if var not in bqm.linear:
            bqm.add_variable(var, bias)
        else:
            bqm.set_linear(var, bqm.get_linear(var) + bias)

    def add_quadratic(u, v, weight):
        if (u,v) not in bqm.quadratic and (v,u) not in bqm.quadratic:
            bqm.add_interaction(u, v, weight)
        else:
            current = bqm.get_quadratic(u,v)
            bqm.set_quadratic(u, v, current + weight)

    # (A) Persönlichkeits-Bias
    for (p, t), bias_val in personality_bias.items():
        var = f"{p}_{t}"
        add_linear(var, -bias_val * scale_factor)

    # (B) Hierarchie-Variablen => baseline 0
    for hv in hierarchy_vars:
        if hv not in bqm.linear:
            bqm.add_variable(hv, 0.0)

    # (C) Kopplungen in der Hierarchie
    # G <-> Epics
    for epic_var in hierarchy_qubits["Epics"]:
        add_quadratic("G", epic_var, -(hierarchy_interaction_strength*scale_factor))

    # Epics <-> Features
    for i, epic_var in enumerate(hierarchy_qubits["Epics"], start=1):
        feat_key = f"Features_E{i}"
        if feat_key in hierarchy_qubits:
            for f_var in hierarchy_qubits[feat_key]:
                add_quadratic(epic_var, f_var, -(epic_feature_interaction_strength*scale_factor))

    # Epics untereinander
    epics_list = hierarchy_qubits["Epics"]
    for i in range(len(epics_list)):
        for j in range(i+1, len(epics_list)):
            add_quadratic(epics_list[i], epics_list[j], -(epic_epic_coupling*scale_factor))

    # Features untereinander
    for epic_id in range(1, 4):
        feat_list = hierarchy_qubits[f"Features_E{epic_id}"]
        for i in range(len(feat_list)):
            for j in range(i+1, len(feat_list)):
                add_quadratic(feat_list[i], feat_list[j], -(feature_feature_coupling*scale_factor))

    # (D) Persönlichkeit <-> Ziel-Hierarchie
    hierarchy_map = {
        "ProjectGoal": hierarchy_qubits["ProjectGoal"],
        "Epics":       hierarchy_qubits["Epics"],
        "Features":    (hierarchy_qubits["Features_E1"]
                        + hierarchy_qubits["Features_E2"]
                        + hierarchy_qubits["Features_E3"])
    }
    for p in team_members:
        for t in big_five_traits:
            trait_val = personality_bias.get((p, t), 0.0)
            for (ctr, clevel), c_strength in personality_hierarchy_couplings.items():
                if (t == ctr) and abs(trait_val)>1e-9:
                    for hv in hierarchy_map[clevel]:
                        add_quadratic(f"{p}_{t}", hv, - (trait_val * c_strength * scale_factor))

    # (E) Persönlichkeit <-> Persönlichkeit
    for (t1,t2), val in personality_personality_couplings.items():
        for (pA, pB) in itertools.combinations(team_members, 2):
            varA = f"{pA}_{t1}"
            varB = f"{pB}_{t2}"
            add_quadratic(varA, varB, -(val*scale_factor))

    return bqm

############################################################
# 3. Uniforme Verteilung über Hierarchie-Bits => Start
############################################################

def uniform_sampleset_hierarchy():
    """
    Erzeuge ein SampleSet mit einer Gleichverteilung ueber
    die 13 Hierarchie-Bits (2^13 = 8192).
    Dabei setzen wir Persoenlichkeitsbits=0.
    => simuliert maximale Unsicherheit nur in der Ziel-Hierarchie.

    Achtung: 2^13=8192 => kann Zeit/RAM beanspruchen,
    aber ist meist noch machbar.
    """
    # sammle 13 Hierarchievariablen
    hvars = hierarchy_vars  # G, E1..E3, F1_1..F3_3 => 13 Bits

    all_samples = []
    all_energies = []
    all_occ = []

    for i in range(2**len(hvars)):
        sdict = {}
        # set hierarchy bits
        for idx, hv in enumerate(hvars):
            bitval = (i>>idx)&1
            sdict[hv] = bitval
        # persona bits => 0
        for (p,t) in personality_bias.keys():
            sdict[f"{p}_{t}"] = 0

        all_samples.append(sdict)
        all_energies.append(0.0)   # dummy
        all_occ.append(1)         # count=1

    ss = SampleSet.from_samples(all_samples, vartype=Vartype.BINARY, energy=all_energies)
    ss.record.num_occurrences = np.array(all_occ, dtype=np.int64)
    return ss

############################################################
# 4. Observational Entropy (klassische)
############################################################

def observational_entropy(sampleset, subset_vars):
    freq = {}
    total = float(sum(sampleset.record.num_occurrences))
    for sample, count in zip(sampleset.samples(), sampleset.record.num_occurrences):
        macro_state = tuple(sample[v] for v in subset_vars)
        freq[macro_state] = freq.get(macro_state, 0) + count

    s=0.0
    for st,c in freq.items():
        p = c/total
        if p>1e-15:
            s -= p*math.log(p)
    return s

############################################################
# 5. Projektphasen-Simulation
############################################################

def run_phase(scale_factor):
    """
    Erzeuge BQM mit scale_factor, laufe auf D-Wave,
    berechne Observational Entropy ueber (Goal,Epics,Features)
    => return (sampleset, S_goal, S_epics, S_features, energies)
    """
    # BQM bauen
    bqm_phase = build_bqm(scale_factor)
    sampler_local = EmbeddingComposite(
        DWaveSampler(
            token='TO FILL IN YOUR TOKEN',
            solver={'name': 'Advantage_system4.1'}
        )
    )
    sset_phase = sampler_local.sample(bqm_phase, num_reads=500, chain_strength=2.0)

    # Measure Entropies
    Sg = observational_entropy(sset_phase, goal_vars)
    Se = observational_entropy(sset_phase, epics_vars)
    Sf = observational_entropy(sset_phase, features_vars)

    # Extrahiere Energien
    energies = list(sset_phase.data_vectors['energy'])

    return sset_phase, (Sg, Se, Sf), energies

# Hauptablauf mit Energieausgabe
scale_factors = [0.1, 0.3, 0.6, 1.0]
for i, sf in enumerate(scale_factors):
    print(f"\n=== PHASE {i+1}: scale_factor={sf} ===")
    sset, (Sg, Se, Sf), energies = run_phase(sf)
    print(f"Goal-Ebene    Entropie: {Sg:.3f} nats (bits={Sg/math.log(2):.3f})")
    print(f"Epics-Ebene   Entropie: {Se:.3f} nats (bits={Se/math.log(2):.3f})")
    print(f"Features-Eb.  Entropie: {Sf:.3f} nats (bits={Sf/math.log(2):.3f})")
    best_ = sset.first
    print(f"BEST SOLUTION in Phase {i+1}: {best_}")

    # Ausgabe der Energien
    print(f"Energien in Phase {i+1}: {energies[:10]} ...")  # Zeigt die ersten 10 Energien



=== PHASE 1: scale_factor=0.1 ===
Goal-Ebene    Entropie: 0.616 nats (bits=0.889)
Epics-Ebene   Entropie: 2.021 nats (bits=2.916)
Features-Eb.  Entropie: 5.596 nats (bits=8.074)
BEST SOLUTION in Phase 1: Sample(sample={'E1': 1, 'E2': 1, 'E3': 0, 'F1_1': 1, 'F1_2': 1, 'F1_3': 1, 'F2_1': 1, 'F2_2': 1, 'F2_3': 1, 'F3_1': 0, 'F3_2': 1, 'F3_3': 0, 'G': 0, 'P1_A': 1, 'P1_C': 0, 'P1_E': 1, 'P1_N': 0, 'P1_O': 0, 'P2_A': 1, 'P2_C': 1, 'P2_E': 1, 'P2_N': 1, 'P2_O': 0, 'P3_A': 0, 'P3_C': 1, 'P3_E': 0, 'P3_N': 0, 'P3_O': 0, 'P4_A': 1, 'P4_C': 0, 'P4_E': 1, 'P4_N': 0, 'P4_O': 0, 'P5_A': 1, 'P5_C': 0, 'P5_E': 1, 'P5_N': 0, 'P5_O': 0, 'P6_A': 0, 'P6_C': 1, 'P6_E': 0, 'P6_N': 1, 'P6_O': 1, 'P7_A': 1, 'P7_C': 0, 'P7_E': 1, 'P7_N': 0, 'P7_O': 1}, energy=-2.4469999999999996, num_occurrences=1, chain_break_fraction=0.0)
Energien in Phase 1: [-2.4469999999999996, -2.1530000000000005, -2.144000000000001, -2.087, -2.0390000000000015, -2.032, -2.013000000000001, -2.0110000000000006, -2.0100000000000007, -2.0